# WSB Strategy

Project description

In [5]:
import bt
import talib as ta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'talib'

In [ ]:
class new_strategy:
    def __init__(self, name, tickers, short, long, start, end=None):
        self.name = name
        self.tickers = tickers
        self.short = short
        self.long = long
        self.start = start
        self.end = end
        self.data = None
        self.sma_short = None
        self.sma_long = None
        self.target_weights = None
        # Acquiring number of tickers to balance weights equally
        self.tickers_number = tickers.count(',') + 1
        
    def get_data(self):
        self.data = bt.get(self.tickers, start=self.start, end=self.end)
        
    def sma_crossover(self, tree_tickers=[]):
        # Obtaining both short and long moving averages to balance weights
        self.sma_short = self.data.rolling(self.short).mean()
        self.sma_long = self.data.rolling(self.long).mean()
        
        # Copying data frame and assigning values when SMAs intersect
        self.target_weights = self.sma_long.copy()
        self.target_weights[self.sma_short > self.sma_long] = 1/self.tickers_number
        self.target_weights[self.sma_short <= self.sma_long] = -(1/self.tickers_number)
        
        # Creating strategy balancing weights as assigned in dataframe
        cross_strategy = bt.Strategy(self.name, [
            bt.algos.WeighTarget(self.target_weights),
            bt.algos.Rebalance()
        ], tree_tickers)
        
        # Running backtest with the ticker's data and strategy
        return bt.Backtest(cross_strategy, self.data)   
    
    def momentum_strategy(self, selectors, period, tree_tickers=[]):
        momentum_strategy = bt.Strategy(self.name, [
            bt.algos.RunDaily() if period=='Daily' else bt.algos.RunMonthly() ,
            bt.algos.SelectAll(),
            bt.algos.SelectMomentum(selectors),
            bt.algos.WeighEqually(),
            bt.algos.Rebalance()
        ], tree_tickers)
        
        return bt.Backtest(momentum_strategy, self.data)
        


### Returning the best strategy based on CAGR

In [ ]:
tickers = [ 'NOK', 'BB', 'GE', 'SRPT', 
            'IQ', 'SPCE', 'RYN', 'MAC', 'AG', 'SNDL', 'WKHS', 
            'TR', 'M', 'HPE', 'WBA', 'SPWR' , 'NVAX', 'BBBY', 
            'TAK', 'TSLA', 'BCRX']

tickers_string = ','.join(tickers)

results = pd.DataFrame(columns=['strategy_name', 'cagr', 'short_sma', 'long_sma', 'sharpe', 'sortino', 'total_return', 'strategy_run'])
results['short_sma'] = results['short_sma'].astype(int)
results['long_sma'] = results['long_sma'].astype(int)

best_strategy = None

for short in range(10,70,5):
    for long in range(80,201,10):
        strategy_name = 'SMA_'+ str(short) + '_' + str(long)
        dynamic_strategy = new_strategy(strategy_name, tickers_string, short, long, '2018-01-01', '2021-06-01')
        dynamic_strategy.get_data()
        backtest = dynamic_strategy.sma_crossover()
        res = bt.run(backtest)
        
        res_cagr = res.stats.at['cagr', strategy_name]
        res_sharpe = res.stats.at['daily_sharpe', strategy_name]
        res_return = res.stats.at['total_return', strategy_name]
                
        results = results.append({
            'strategy_name': strategy_name,
            'cagr': res_cagr,
            'short_sma': short,
            'long_sma': long,
            'sharpe': res_sharpe,
            'total_return': res_return,
            'strategy_run': res
        }, ignore_index = True)
        
        if best_strategy == None or best_strategy.stats.at['cagr', best_strategy.stats.columns[0]] <= res_cagr:
            best_strategy = res
            
best_strategy.display()
best_strategy.plot()

In [ ]:
best_strategy.plot_security_weights()

In [ ]:
cagr_df = results.pivot(index='short_sma', columns='long_sma', values='cagr')
fig, ax = plt.subplots(figsize=(25,15))
ax = sns.heatmap(
    data = cagr_df,
    annot = True,
    annot_kws = {'fontsize': 15},
    linewidths = 0.5,
    cmap = 'cividis'
)

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=15)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=15)

ax.set_xlabel('Long SMA', fontsize = 15)
ax.set_ylabel('Short SMA', fontsize = 15)
ax.set_title('CAGR per SMA Combination', fontsize = 23)

plt.show()

In [ ]:
cagr_df = results.pivot(index='short_sma', columns='long_sma', values='sharpe')
fig, ax = plt.subplots(figsize=(25,15))
ax = sns.heatmap(
    data = cagr_df,
    annot = True,
    annot_kws = {'fontsize': 15},
    linewidths = 0.5,
    cmap = 'magma'
)

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=15)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=15)

ax.set_xlabel('Long SMA', fontsize = 15)
ax.set_ylabel('Short SMA', fontsize = 15)
ax.set_title('Sharpe ratio per SMA Combination', fontsize = 23)

plt.show()

In [ ]:
cagr_df = results.pivot(index='short_sma', columns='long_sma', values='total_return')
fig, ax = plt.subplots(figsize=(25,15))
ax = sns.heatmap(
    data = cagr_df,
    annot = True,
    annot_kws = {'fontsize': 15},
    linewidths = 0.5,
    cmap = 'plasma'
)

ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=15)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=15)

ax.set_xlabel('Long SMA', fontsize = 15)
ax.set_ylabel('Short SMA', fontsize = 15)
ax.set_title('Total return per SMA Combination', fontsize = 23)

plt.show()

## Bollinger Bands


In [ ]:
#Variables
WSBI = [ 'NOK', 'BB', 'GE', 'SRPT', 
            'IQ', 'SPCE', 'RYN', 'MAC', 'AG', 'SNDL', 'WKHS', 
            'TR', 'M', 'HPE', 'WBA', 'SPWR' , 'NVAX', 'BBBY', 
            'TAK', 'TSLA', 'BCRX']

data = bt.get(tickers=WSBI, start= '2018-01-01', end='2021-06-01')

upper_bands = []
middle_bands = []
lower_bands = []

for ticker in data.columns:

    # Bollinger Band Calculation
    upper, middle, lower  = ta.BBANDS(
            data[ticker],       # Data
            timeperiod = 20,    # n Period simple moving average
            nbdevup    = 2,     # k Standard deviation upwards  
            nbdevdn    = 2)     # k Standard deviation downwards

    upper_bands.append(upper)
    middle_bands.append(middle)
    lower_bands.append(lower)
    
# Create placeholder for signal
lower_bands_df = pd.DataFrame()

# Copy over lower bands for our tickers and transpose the dataframe
lower_bands_df = pd.DataFrame.from_records(data = lower_bands).T

# Rename columns to match our adjustced close dataframe
lower_bands_df.columns = data.columns

# Define signal as True when adjust close is less than lower band
signal = data < lower_bands_df

strategy = bt.Strategy('Bollinger Bands',
                       [bt.algos.SelectWhere(signal),         
                        bt.algos.WeighEqually(),
                        bt.algos.Rebalance()])

# Create the Backtest algorithm using the FANG historical adjusted close
backtest_bbands = bt.Backtest(strategy, data)

# Run the backtest and store the results
results_BB = bt.run(backtest_bbands)

# Plot equity progression
results_BB.plot()

results_BB.display()

results_BB_MA = bt.run(backtest_bbands, prof, wsb)

results_BB_MA.plot()